# Python 機器學習從零至一 

> 正規方程

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from pyvizml import NormalEquation

## 關於數值預測的任務

## （複習）數值預測的任務：迴歸模型

- 「數值預測」是「監督式學習」的其中一種應用類型。
- 預測的目標向量 $y$ 屬於連續型數值變數。
- 更常被稱為「迴歸模型」。

## （複習）預測 NBA 球員的體重

- 資料（Experience）：一定數量的球員資料。
- 任務（Task）：利用模型預測球員的體重。
- 評估（Performance）：模型預測的體重與球員實際體重的誤差大小。
- 但書（Condition）：隨著資料觀測值筆數增加，預測誤差應該要減少。

In [2]:
# players_stats 資料中的 weightKilograms
csv_url = "https://raw.githubusercontent.com/yaojenkuo/ml-newbies/master/player_stats.csv"
player_stats = pd.read_csv(csv_url)
y = player_stats["weightKilograms"].values
y.dtype

dtype('float64')

## 該如何預測 NBA 球員的體重

- 隨意猜測的虛假模型。
- 基於規則（Rule-based）的專家模型。
- 基於最小化損失函數的機器學習模型。
- ...等。

## 基於最小化損失函數的機器學習模型

將 `heightMeters` 當作特徵矩陣 $x_i$ 作為體重的預測依據。

\begin{equation}
\operatorname*{arg\,min}_w \; \frac{1}{m}\sum_{i}^{m}{(y^{(train)}_i - \hat{y_i}^{(train)})^2} = \frac{1}{m}\sum_{i}^{m}{(y^{(train)}_i - x_i^{(train)} w)^2}
\end{equation}

In [3]:
X = player_stats["heightMeters"].values.reshape(-1, 1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)
h = LinearRegression()
h.fit(X_train, y_train)
y_hat = h.predict(X_valid)

## 評估基於最小化損失函數的機器學習模型：均方誤差

\begin{equation}
\text{MSE}_{valid} = \frac{1}{m}\sum_{i}^{m}{(y^{(valid)}_i - \hat{y_i}^{(valid)})^2}
\end{equation}

In [4]:
mse_ml = np.sum((y_valid - y_hat)**2) / y_valid.size
mse_ml

62.807098266825335

## 如何決定 `X_train` 與 `y_train` 之間的關聯 $w$

Scikit-Learn 預測器的關鍵：`fit()` 方法：正規方程（Normal equation）

## 正規方程

## 使用基於最小化損失函數的機器學習模型解決數值預測的任務

創造一個 $h$ 函數可以將無標籤資料 $x$ 作為輸入，以係數 $w$ 相乘後輸出 $\hat{y}$

\begin{equation}
\hat{y} = h(x; w) = w_0 + w_1x_1 + ... + w_nx_n
\end{equation}

## 寫作成向量矩陣相乘形式，為 $w_0$ 補上 $x_0=1$

\begin{align}
\hat{y} &= w_0x_0 + w_1x_1 + ... + w_nx_n, \; where \; x_0 = 1 \\
&= w^Tx
\end{align}

## 向量矩陣相乘形式

- $m + 1$ 為觀測值列數。
- $n + 1$ 是特徵個數。

\begin{equation}
\hat{y} = h(X; w) = 
\begin{bmatrix} x_{00}, x_{01}, ..., x_{0n} \\ x_{10}, x_{11}, ..., x_{1n} \\.\\.\\.\\ x_{m0}, x_{m1}, ..., x_{mn}
\end{bmatrix}
\begin{bmatrix} w_0 \\ w_1 \\.\\.\\.\\ w_n \end{bmatrix} = Xw
\end{equation}

## $h(X; w)$ 是基於 $w$ 的函數

- 如果第 $i$ 個特徵 $x_i$ 對應的係數 $w_i$ 為正數，該特徵與 $\hat{y}$ 的變動同向。
- 如果第 $i$ 個特徵 $x_i$ 對應的係數 $w_i$ 為負數，該特徵與 $\hat{y}$ 的變動反向。
- 如果第 $i$ 個特徵 $x_i$ 對應的係數 $w_i$ 為零，該特徵對 $\hat{y}$ 的變動沒有影響。

## 資料與任務已經被定義妥善

- 特徵矩陣 $X$
- 目標向量 $y$
- 係數向量 $w$
- 任務：將 $X$ 輸入 $h$ 來預測 $\hat{y}$

## 定義評估

評估 $h$ 的方法是計算 $y^{(train)}$ 與 $\hat{y}^{(train)}$ 之間的均方誤差（Mean squared error）。

\begin{equation}
\operatorname*{arg\,min}_w \; \frac{1}{m}\sum_{i}^{m}{(y^{(train)}_i - \hat{y_i}^{(train)})^2}
\end{equation}

## 寫為向量運算的外觀

\begin{equation}
\operatorname*{arg\,min}_w \; \frac{1}{m} \parallel {y^{(train)}_i - X^{(train)}w \parallel^2}
\end{equation}

## 將均方誤差表達為一個基於係數向量 $w$ 的損失函數 $J(w)$

\begin{equation}
J(w) = \frac{1}{m} \parallel {y^{(train)}_i - X^{(train)}w \parallel^2}
\end{equation}

## 整理一下 $J(w)$ 的外觀

為了書寫方便，我們省略訓練資料的註記$(train)$。

\begin{align}
J(w) &= \frac{1}{m}(Xw - y)^T(Xw - y) \\
&= \frac{1}{m}(w^TX^T - y^T)(Xw - y) \\
&= \frac{1}{m}(w^TX^TXw - w^TX^Ty - y^TXw + y^Ty) \\
&= \frac{1}{m}(w^TX^TXw - (Xw)^Ty - y^TXw + y^Ty) \\
&= \frac{1}{m}(w^TX^TXw - 2(Xw)^Ty + y^Ty)
\end{align}

## 求解 $J(w)$ 斜率為零的位置 $w^*$

\begin{gather}
\frac{\partial}{\partial w} J(w) = 0 \\
2X^TXw - 2X^Ty = 0 \\
X^TXw = X^Ty \\
w^* = (X^TX)^{-1}X^Ty
\end{gather}

## $w^*$ 求解稱為「正規方程」

\begin{equation}
w^* = (X^{(train)T}X^{(train)})^{-1}X^{(train)T}y^{(train)}
\end{equation}

## 自行定義正規方程類別 NormalEquation

```python
class NormalEquation:
    """
    This class defines the Normal equation for linear regression.
    Args:
        fit_intercept (bool): Whether to add intercept for this model.
    """
    def __init__(self, fit_intercept=True):
        self._fit_intercept = fit_intercept
```

```python
    def fit(self, X_train, y_train):
        """
        This function uses Normal equation to solve for weights of this model.
        Args:
            X_train (ndarray): 2d-array for feature matrix of training data.
            y_train (ndarray): 1d-array for target vector of training data.
        """
        self._X_train = X_train.copy()
        self._y_train = y_train.copy()
        m = self._X_train.shape[0]
        if self._fit_intercept:
            X0 = np.ones((m, 1), dtype=float)
            self._X_train = np.concatenate([X0, self._X_train], axis=1)
        X_train_T = np.transpose(self._X_train)
        left_matrix = np.dot(X_train_T, self._X_train)
        right_matrix = np.dot(X_train_T, self._y_train)
        left_matrix_inv = np.linalg.inv(left_matrix)
        w = np.dot(left_matrix_inv, right_matrix)
        w_ravel = w.ravel().copy()
        self._w = w
        self.intercept_ = w_ravel[0]
        self.coef_ = w_ravel[1:]
```

```python
    def predict(self, X_test):
        """
        This function returns predicted values with weights of this model.
        Args:
            X_test (ndarray): 2d-array for feature matrix of test data.
        """
        self._X_test = X_test.copy()
        m = self._X_test.shape[0]
        if self._fit_intercept:
            X0 = np.ones((m, 1), dtype=float)
            self._X_test = np.concatenate([X0, self._X_test], axis=1)
        y_pred = np.dot(self._X_test, self._w)
        return y_pred
```

In [5]:
h_sklearn = LinearRegression()
h_sklearn.fit(X_train, y_train)
h_ne = NormalEquation()
h_ne.fit(X_train, y_train)

In [6]:
print(h_sklearn.intercept_) # 截距項
print(h_sklearn.coef_)      # 係數項
print(h_ne.intercept_)      # 截距項
print(h_ne.coef_)           # 係數項

-95.14864145823769
[97.25416437]
-95.1486414580504
[97.25416437]


In [7]:
# 預測
y_hat = h_sklearn.predict(X_valid)
y_hat[:5]

array([ 95.46952071,  95.46952071,  92.55189578, 107.14002044,
        97.414604  ])

In [8]:
# 預測
y_hat = h_ne.predict(X_valid)
y_hat[:5]

array([ 95.46952071,  95.46952071,  92.55189578, 107.14002044,
        97.414604  ])

## 重點統整

- 如何決定 `X_train` 與 `y_train` 之間的關聯 $w$：正規方程（Normal equation）
- 將 $h(X; w)$ 寫作成向量矩陣相乘形式，為 $w_0$ 補上 $x_0=1$
- 求解 $J(w)$ 斜率為零的位置 $\frac{\partial}{\partial w} J(w) = 0$
- 正規方程求解 $w^* = (X^TX)^{-1}X^Ty$